# We will use utilization data here
The goal is to use current utilization data to pull historical claims for the same patient

In [27]:
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path
from dotenv import load_dotenv
import glob

from dotenv import load_dotenv

dir_path = Path.cwd()
data_path = (dir_path / "data").resolve()
temp_path = (dir_path / "temp").resolve()
env_path = (dir_path / ".env").resolve()

load_dotenv(env_path)

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option("display.max_rows", 40)

CSV data  
Let's merge all our utilization data here

In [28]:
df = pd.read_csv("data.csv",  encoding='ISO 8859-1')
df['Service Date'] = pd.to_datetime(df['Service Date'])

# Sort DataFrame by 'Member ID' and 'Service Date' in descending order
df.sort_values(by=['Member ID', 'Service Date'], ascending=[True, False], inplace=True)
df.columns = df.columns.str.lower()
df['member id'] = df['member id'].astype(str)
df['policy'] = df['procedure code'] + ": " + df['procedure guidelines']
df.drop(['procedure guidelines'], axis=1, inplace=True)
df.head()

,member id,member first name,member last name,tooth surface,tooth nbr,provider,npi,service date,procedure code,procedure description,submitted amount,approved amount,allowed amount,network savings,deductible amt,cov %,patient owes,plan payment,policy
0,1TA9YY3MK99,Evaleen,Gaitung,NaN,NaN,Dr. Smith,5WC2CU9KQ89,2024-04-01,D0150,"Comprehensive oral evaluation, new or establis...",75,55,55,20,0,100,0,55,"D0150: Once per lifetime, per dentist/provider..."
1,1TA9YY3MK99,Evaleen,Gaitung,NaN,NaN,Dr. Smith,5WC2CU9KQ89,2024-04-01,D1110,"Prophylaxis  adult Removal of plaque, calcul...",90,70,70,20,0,100,0,70,D1110: Twice per calendar year age thirteen (1...
2,1TA9YY3MK99,Evaleen,Gaitung,NaN,NaN,Dr. Smith,5WC2CU9KQ89,2024-04-01,D0274,Bitewings  four (4) radiographic images,100,80,80,20,0,100,0,80,D0274: Once per benefit period. Plan benefits ...
3,1TA9YY3MK99,Evaleen,Gaitung,NaN,NaN,Dr. Smith,5WC2CU9KQ89,2024-04-01,D4341,Periodontal Scaling and Root planning (SRP) 4 ...,400,300,300,100,0,100,0,300,D4341: One (1) per quadrant per twenty-four (2...
4,1TA9YY3MK99,Evaleen,Gaitung,NaN,NaN,Dr. Smith,5WC2CU9KQ89,2024-04-01,D4355,Full mouth debridement to enable a comprehensi...,250,200,200,50,0,100,0,200,"D4355: 1 per 36 months, age 18 and older. Not ..."


## Prompting

Now let's append the current claim to historical utilization for each member  
- We want the claim, find it's member, then use the member_id to pull historical claims

In [29]:
from tabulate import tabulate # first, we'll convert to markdown

In [30]:
# Source code (modified slightly) from 'laura-xy-lee' from: https://tinyurl.com/3fr42jky
def convert_to_md(df):
        # Convert table to markdown
        md = tabulate(df, tablefmt='github', headers='keys', showindex=False)
        return md

Since we have a mix of claim types (clinically review with & w/o issue and not reviewed), let's randomly shuffle the df just in case the LLM is preprogrammed to answer a certain due to a previous api call

In [31]:
for member_id, group_df in df.groupby('member id'):
    md = convert_to_md(group_df)
    print(f"## Member ID: {member_id}\n\n{md}\n")

## Member ID: 1TA9YY3MK99

| member id   | member first name   | member last name   |   tooth surface |   tooth nbr | provider   | npi         | service date        | procedure code   | procedure description                                                                                       |   submitted amount |   approved amount |   allowed amount |   network savings |   deductible amt |   cov % |   patient owes |   plan payment | policy                                                                                                                                                                                                          |
|-------------|---------------------|--------------------|-----------------|-------------|------------|-------------|---------------------|------------------|-------------------------------------------------------------------------------------------------------------|--------------------|-------------------|------------------|-------------------|---

Let's bring in our policy

In [32]:
base_prompt = """You are the world's foremost investigator into fraud, waste, and abuse in the dental insurance industry.
Here are our definitions for FWA:
###
Fraud: Fraud hinges on intent: healthcare workers knowingly working outside of accepted standards for their own financial gain. Some common fraud examples include billing for services not rendered, misrepresenting dates of service, diagnosing unnecessary or incorrect treatment, or misrepresenting the identity of the patient. While these aren’t limited to dentistry, spotting nefarious intent within inappropriate claims must be done right away before unwarranted payment escalates.
Abuse: Abuse typically involves acts that are inconsistent with identified acceptable and legitimate practices. This can result in additional and unnecessary costs to the payer, as well as possible injury to patients, who may suffer from a lack of needed care, incorrect care, or deficient care as a result. Over-coding procedures falls under the category of abuse. Examples of over-coding include billing for a higher extraction code when the lower code is more in line with the actual process per extraction codes contained in CDT. The higher code is generally reimbursed at a higher rate, thus using more of the patient's available dental plan benefit.
Waste: A major problem for dental payers that often goes unchecked is waste, or billing for medically unnecessary services. Waste in dental services not only affects the dental plans, but it inaccurately deducts from their members’ yearly maximum benefit, potentially leaving insufficient funds for other necessary procedures.
###

"""
print(base_prompt)

You are the world's foremost investigator into fraud, waste, and abuse in the dental insurance industry.
Here are our definitions for FWA:
###
Fraud: Fraud hinges on intent: healthcare workers knowingly working outside of accepted standards for their own financial gain. Some common fraud examples include billing for services not rendered, misrepresenting dates of service, diagnosing unnecessary or incorrect treatment, or misrepresenting the identity of the patient. While these aren’t limited to dentistry, spotting nefarious intent within inappropriate claims must be done right away before unwarranted payment escalates.
Abuse: Abuse typically involves acts that are inconsistent with identified acceptable and legitimate practices. This can result in additional and unnecessary costs to the payer, as well as possible injury to patients, who may suffer from a lack of needed care, incorrect care, or deficient care as a result. Over-coding procedures falls under the category of abuse. Example

In [33]:
additional_prompt = """###
...
###
"""
print(additional_prompt)

###

###



In [34]:
few_shot_prompt = """Example response with a severity score (none, low, medium, high):
###
XXXXXX
###
"""
print(few_shot_prompt)

Example response with a severity score (none, low, medium, high):
###
XXXXXX
###



In [35]:
# Initialize an empty list to store results
prompt_list = []

unique_ids = list(np.unique(df['member id']))

for id in unique_ids:
    member_id = id
    _util_history = df[df['member id']==member_id]
    prompt = f"{base_prompt}\nHere's the member history in markdown format:\n###\n{convert_to_md(_util_history)}\n###\n"

    prompt +=f"What are your thoughts about this claim, using ONLY the policy in the context, are there signs of fraud, waste, abuse, educational opportunities, upcoding, unbundling, issues with the treatment sequence, etc? ENSURE to add a severity score (none, low, medium, high) and reference the line number when appropriate. Call out potential savings in bold **text** not markdown based on your findings"

    # print(prompt)
    prompt_list.append([member_id, prompt])


In [36]:
prompt_df = pd.DataFrame(prompt_list, columns=['member_id', 'prompt'])
prompt_df

,member_id,prompt
0,1TA9YY3MK99,You are the world's foremost investigator into...
1,5E73XN8XG05,You are the world's foremost investigator into...
2,5KN2RV8RX93,You are the world's foremost investigator into...


In [37]:
print(prompt_df.tail(1)['prompt'].values[0])

You are the world's foremost investigator into fraud, waste, and abuse in the dental insurance industry.
Here are our definitions for FWA:
###
Fraud: Fraud hinges on intent: healthcare workers knowingly working outside of accepted standards for their own financial gain. Some common fraud examples include billing for services not rendered, misrepresenting dates of service, diagnosing unnecessary or incorrect treatment, or misrepresenting the identity of the patient. While these aren’t limited to dentistry, spotting nefarious intent within inappropriate claims must be done right away before unwarranted payment escalates.
Abuse: Abuse typically involves acts that are inconsistent with identified acceptable and legitimate practices. This can result in additional and unnecessary costs to the payer, as well as possible injury to patients, who may suffer from a lack of needed care, incorrect care, or deficient care as a result. Over-coding procedures falls under the category of abuse. Example

In [38]:
# prompt_df.to_csv(data_path / 'prompts.csv', index=False)

# Hard to save as csv due to markdown and many delimiters in the 'prompt' column
# Specify the file path
file_path = 'prompts.npy'

# Save the DataFrame to a NumPy binary file
np.save(file_path, prompt_df)

In [39]:
# To load
# Load the DataFrame back from the binary file
prompt_df_loaded = np.load(file_path, allow_pickle=True)
prompt_df_loaded = pd.DataFrame(prompt_df_loaded, columns=prompt_df.columns)
prompt_df_loaded

,member_id,prompt
0,1TA9YY3MK99,You are the world's foremost investigator into...
1,5E73XN8XG05,You are the world's foremost investigator into...
2,5KN2RV8RX93,You are the world's foremost investigator into...
